<a href="https://colab.research.google.com/github/crisbpadilla/DataScience-M2M-course-Datatalent/blob/main/Widget_withBokeh_excercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from bokeh.io import output_notebook, push_notebook
from bokeh.plotting import figure, show
from bokeh.palettes import cividis
from bokeh.models import CrosshairTool

import pandas as pd

output_notebook()

columns=['year','state','candidate','party','candidatevotes','totalvotes']
df = pd.read_csv('1976-2016-president.csv',usecols=columns,dtype={'year':str})
state_group= df.groupby('state')
year=df['year'].unique()

color=cividis(len(state_group))
p=figure(tooltips= '$name @x:@y%',x_range= year)

state_line={}

for i,states in enumerate(state_group.groups):

  data=state_group.get_group(states)
  party_data=data[data['party']=='democrat']
  percentage=party_data['candidatevotes']/party_data['totalvotes']*100
  line= p.line(x=party_data['year'],y=percentage, color=color[i], name= states)

  state_line[states]=line

p.add_tools(CrosshairTool())
show(p)

In [5]:
for line in state_line.values():
  line.visible= False
show(p)
print(state_line)

{'Alabama': GlyphRenderer(id='p1641', ...), 'Alaska': GlyphRenderer(id='p1650', ...), 'Arizona': GlyphRenderer(id='p1659', ...), 'Arkansas': GlyphRenderer(id='p1668', ...), 'California': GlyphRenderer(id='p1677', ...), 'Colorado': GlyphRenderer(id='p1686', ...), 'Connecticut': GlyphRenderer(id='p1695', ...), 'Delaware': GlyphRenderer(id='p1704', ...), 'District of Columbia': GlyphRenderer(id='p1713', ...), 'Florida': GlyphRenderer(id='p1722', ...), 'Georgia': GlyphRenderer(id='p1731', ...), 'Hawaii': GlyphRenderer(id='p1740', ...), 'Idaho': GlyphRenderer(id='p1749', ...), 'Illinois': GlyphRenderer(id='p1758', ...), 'Indiana': GlyphRenderer(id='p1767', ...), 'Iowa': GlyphRenderer(id='p1776', ...), 'Kansas': GlyphRenderer(id='p1785', ...), 'Kentucky': GlyphRenderer(id='p1794', ...), 'Louisiana': GlyphRenderer(id='p1803', ...), 'Maine': GlyphRenderer(id='p1812', ...), 'Maryland': GlyphRenderer(id='p1821', ...), 'Massachusetts': GlyphRenderer(id='p1830', ...), 'Michigan': GlyphRenderer(id=

In [6]:
from ipywidgets import interact,SelectMultiple,IntRangeSlider

In [7]:
from bokeh.io import output_notebook, push_notebook


handle = show(p, notebook_handle=True)
default_value=['Hawaii','Washington']

select_line= SelectMultiple( options=state_line.keys(),
                            rows=10, description='Select state(s):',
                             value= default_value)

@interact(states=select_line)
def update(states):
  for state_line_name,state_line_fig in state_line.items():
    if state_line_name in states:
      state_line_fig.visible= True
    else:
      state_line_fig.visible= False
  push_notebook(handle=handle)

#set up the range slider

#first create the object that contain the atributes of the range slider
min_year=int(year[0])
max_year=int(year[-1])

select_range= IntRangeSlider(value= [min_year,max_year],
                             min=min_year,max=max_year,step=4,
                             description='Year range')

@interact(years=select_range)#This is a tupple value ex:(california,colorado)
def update(years):
    x_years = range(years[0], years[1] + 1, 4)
    p.x_range.factors = list(map(str, x_years))#factors is how bokeh represents its data
    push_notebook(handle=handle)#rerender the plot with the changes
                                #that we had made



interactive(children=(SelectMultiple(description='Select state(s):', index=(11, 47), options=('Alabama', 'Alas…

interactive(children=(IntRangeSlider(value=(1976, 2016), description='Year range', max=2016, min=1976, step=4)…

In [8]:
print(state_line)
# print(year[-1])

{'Alabama': GlyphRenderer(id='p1641', ...), 'Alaska': GlyphRenderer(id='p1650', ...), 'Arizona': GlyphRenderer(id='p1659', ...), 'Arkansas': GlyphRenderer(id='p1668', ...), 'California': GlyphRenderer(id='p1677', ...), 'Colorado': GlyphRenderer(id='p1686', ...), 'Connecticut': GlyphRenderer(id='p1695', ...), 'Delaware': GlyphRenderer(id='p1704', ...), 'District of Columbia': GlyphRenderer(id='p1713', ...), 'Florida': GlyphRenderer(id='p1722', ...), 'Georgia': GlyphRenderer(id='p1731', ...), 'Hawaii': GlyphRenderer(id='p1740', ...), 'Idaho': GlyphRenderer(id='p1749', ...), 'Illinois': GlyphRenderer(id='p1758', ...), 'Indiana': GlyphRenderer(id='p1767', ...), 'Iowa': GlyphRenderer(id='p1776', ...), 'Kansas': GlyphRenderer(id='p1785', ...), 'Kentucky': GlyphRenderer(id='p1794', ...), 'Louisiana': GlyphRenderer(id='p1803', ...), 'Maine': GlyphRenderer(id='p1812', ...), 'Maryland': GlyphRenderer(id='p1821', ...), 'Massachusetts': GlyphRenderer(id='p1830', ...), 'Michigan': GlyphRenderer(id=